## RAG pipeline 
Команда Утики MISIS

In [1]:
import torch
import numpy as np
import pandas as pd
import re
import torch
import nltk
import joblib
import pandas as pd
from IPython.display import display, Markdown

from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
from transformers import AutoModelForSequenceClassification
from transformers import set_seed
from sentence_transformers import SentenceTransformer

from sklearn.neighbors import NearestNeighbors

from utils.utils import Chunker, Generator
from utils.spell_checker import SpellChecker
from utils.llm_config import GenerationConfig
from utils.toxicity_classifier import ToxicityClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Загружаем модель ретривера и модель генератора

In [2]:
retriever_model = SentenceTransformer("BAAI/bge-m3", device='cuda')
retriever_model.max_seq_length = 512

generator_model_name = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_model_name).cuda().eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Загружаем данные, тут формат работы в ноутбуке, этот пункт будет заменен на обращение к ClickHouse

In [3]:
texts = pd.read_csv('data/documents.csv', header=None, names=['url', 'text'])
texts = texts.dropna()
texts['text'] = texts['text'].apply(lambda x: re.sub(' +', ' ', x))
texts['text'] = texts['text'].apply(lambda x: re.sub('\n+', '\n', x))
texts.url = texts.url.apply(lambda x: x if  x[-1] == '/' else x+'/')
texts.text = texts.text.apply(lambda x: x.replace(x[x.find('<!--'): x.rfind('-->')+3], '').replace('Complex', '').strip('\n '))
texts = texts.set_index('url')

#### Инициализируем функцию, которая будет бить наши документы на чанки. Инициализируем генератор.

In [4]:
chunker = Chunker(max_chunk_len=2500, overlap_len=500)
generator = Generator(generator_tokenizer, generator_model, config=GenerationConfig)

In [5]:
chunks, urls = chunker.split_texts(texts)
chunks, urls = np.array(chunks), np.array(urls)

len(chunks)

35091

#### Либо грузим готовые эмбединги, либо делаем их сами

In [6]:
# embeddings = model.encode(chunks, batch_size=32, normalize_embeddings=True, show_progress_bar=True)
# joblib.dump(embeddings, 'bge-m3-new-clear.pkl')

embeddings = joblib.load('bge-m3-new-clear.pkl')
embeddings.shape

(35230, 1024)

#### Инициализируем KNN 

In [7]:
knn = NearestNeighbors(metric='cosine')
knn.fit(embeddings)

NearestNeighbors(metric='cosine')

### Загружаем бенчмарк вопросы

In [6]:
queries = pd.read_csv('data/benchmarks.csv', sep=';')
queries.site_id = queries.site_id.apply(lambda x: x if  x[-1] == '/' else x+'/')
queries.site_id = queries.site_id.apply(lambda x: x if 'http://pravo.gov.ru' in x else 'https://cbr.ru' + x)

In [7]:
question = queries.question[6]
question

'Что такое ВЕБ.РФ?'

### Тестируем

In [10]:
def retrieve(question, model, knn, chunks, urls):
    skores, neighbors = knn.kneighbors(model.encode(question, batch_size=32, normalize_embeddings=True).reshape(1, -1), 3)
    neighbors = neighbors.squeeze()
    retrieved_texts = list(chunks[neighbors])
    retrieved_urls = list(set(urls[neighbors]))
    return retrieved_texts, retrieved_urls

In [11]:
retrieved_texts, retrieved_urls = retrieve(question, retriever_model, knn, chunks, urls)

In [12]:
display(Markdown(generator.get_answer(question, retrieved_texts, retrieved_urls, temperature=0.8)))

Государственная корпорация развития "ВЭБ.РФ".

Использованные документы:
1) http://pravo.gov.ru/proxy/ips/?docbody=&nd=102114195/ 


In [14]:
%timeit generator.get_answer(question, retrieved_texts, retrieved_urls, temperature=0.8)

1.08 s ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Наши крутые фичи!

#### HyDE - Hypothetical Document Embeddings
Использовать перед подачей в ретривер, но просим ответить все равно на изначальный вопрос

In [25]:
queries.question[7]

'Какие полномочия у председателя ВЭБ.РФ?'

In [24]:
generator.hyde(queries.question[7], temperature=1, num_beams=2)

'Полномочия председателя ВЭБ.РФ'

#### Spell Checker
Исправляем ошибки введенного текста

In [26]:
SPELL_CHECKER_MODEL_NAME = 'UrukHan/t5-russian-spell'

spell_checker_tokenizer = T5TokenizerFast.from_pretrained(SPELL_CHECKER_MODEL_NAME)
spell_checker_model = AutoModelForSeq2SeqLM.from_pretrained(SPELL_CHECKER_MODEL_NAME).to(device)

In [27]:
spell_checker = SpellChecker(spell_checker_tokenizer, spell_checker_model)

In [28]:
spell_checker.get_answer('а чт таке конституц росииск федерации')

['А что такое Конституция Российской Федерации?']

#### Question Generation
Генерируем вопросы для каждого чанка, и в ретривер складываем эмбеддинги этих вопросов для улучшения поиска

In [15]:
chunks[0]

'РОССИЙСКАЯ ФЕДЕРАЦИЯ ФЕДЕРАЛЬНЫЙ ЗАКОН О внесении изменений в Федеральный закон "О противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма" в целях совершенствования обязательного контроля Принят Государственной Думой 7 июля 2020 годаОдобрен Советом Федерации 8 июля 2020 года Статья 1 Внести в Федеральный закон от 7 августа 2001 года No 115-ФЗ "О противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма" (Собрание законодательства Российской Федерации, 2001, No 33, ст. 3418; 2002, No 30, ст. 3029; No 44, ст. 4296; 2004, No 31, ст. 3224; 2006, No 31, ст. 3446, 3452; 2007, No 16, ст. 1831; No 31, ст. 3993, 4011; No 49, ст. 6036; 2009, No 23, ст. 2776; 2010, No 30, ст. 4007; No 31, ст. 4166; 2011, No 27, ст. 3873; No 46, ст. 6406; 2012, No 30, ст. 4172; 2013, No 26, ст. 3207; No 44, ст. 5641; No 52, ст. 6968; 2014, No 19, ст. 2315, 2335; No 23, ст. 2934; No 30, ст. 4214, 4219; 2015, No 1, ст

In [17]:
generator.generate_question(chunks[0], temperature=1, num_beams=2)

'Что не подлежит обязательному контролю в соответствии с Федеральным законом от 7 августа 2001 года No 115-ФЗ "О противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма"?'

#### Toxicity detection
Проверяет введенный вопрос на токсичность


In [30]:
TOXICITY_DETECTION_MODEL_NAME = 'cointegrated/rubert-tiny-toxicity'
toxicity_detection_tokenizer = AutoTokenizer.from_pretrained(TOXICITY_DETECTION_MODEL_NAME)
toxicity_detection_model = AutoModelForSequenceClassification.from_pretrained(TOXICITY_DETECTION_MODEL_NAME).to(device)

In [31]:
toxicity_detection_model = ToxicityClassifier(toxicity_detection_tokenizer,
                                              toxicity_detection_model)

In [38]:
normal_question = 'Что такое ВЕБ.РФ?'
toxic_question = 'Что за дичь такое ВЕБ.РФ'
toxicity_detection_model.is_toxic(normal_question), toxicity_detection_model.is_toxic(toxic_question)

('not toxic', 'toxic')